# Association of variables with tumor outcomes

* Univariate analysis of association of radiomics features with locoregional recurrences and disease-free survival. 
* Approach described in Vallieres et al., *Radiomics Strategies for risk assessment of tumor failure in head-and-neck cancer* using a $10 \%$ false discovery rate.

A [tutorial](https://www.statisticshowto.datasciencecentral.com/benjamini-hochberg-procedure/) on the Benjamini-Hochberg procedure. 

**Ranking algorithm:**

1. Sort p-values in ascending order.
2. Assign ranks to p-values.
3. Calculate each individual p-value’s Benjamini-Hochberg critical values asscording to (i/m)Q, where i: rank of p-value, m: the total number of tests, Q: false discovery rate.  
4. Compare the original p-values to the critical B-H values B-H. 

All variables with a p-value < corresponding B-H critical value

**Results Vallieres**
* $0\%$ of PET radiomics features significantly associated with locoregional recurrences.
* $0\%$ of CT radiomics features significantly associated with locoregional recurrences.
* $12\%$ of PET radiomics features significantly associated with overall survival.
* $34\%$ of CT radiomics features significantly associated with overall survival.
* Strongest feature association (both PET and CT) to locoregional recurrence: CT LZHGE_{GLSZM} ($r_s=-0.15$, $p=0.007$).
* Strongest feature association (both PET and CT) to overall survival: CT GLV_{GLRLM} $r_s=0.24$, $p=4\cdot 10^{-5}$).

**NOTE**
* Include algorithm in thesis appendix.

In [47]:
import numpy as np
import pandas as pd

from scipy import stats 
from operator import itemgetter

In [117]:
def spearmans_rank(X, y, fdr=0.1):
    """Assess the association between features and a target 
    variable with Speraman's rank correlations. Includes 
    Benjamini–Hochberg correction.
    
    Args:
        X (array-like): Predictor variables.
        y (array-like): Target variable.
        
    Returns:
        (dict): Key-value paris of feature indicators with statistically 
            significant correlation to the target variable, and the 
            corresponding correlation coefficient and p-values.
            
    """
    _, num_feats = np.shape(X)
    # Compute p-value for each feature.
    pvals, rhos = [], []
    for num in range(num_feats):
        rho, pval = stats.spearmanr(X.values[:, num], y)
        pvals.append(pval), rhos.append(rho)
    
    # Sorted p-values in ascending order.
    sorted_pvals = sorted(pvals)

    sign_feats = {}
    for num, pval in enumerate(sorted_pvals):
        bh_crit = (num + 1) * fdr / num_feats
        if bh_crit < pval:
            sign_feats[X.columns[num]] = (pval, rhos[num])
    
    return sign_feats

In [118]:
# Target variables.
dfs = pd.read_csv('./../../data/to_analysis/target_dfs.csv', index_col=0)
lrr = pd.read_csv('./../../data/to_analysis/target_lrr.csv', index_col=0)

## Image Variables

In [119]:
img_vars = pd.read_csv('./../../data/to_analysis/img_features.csv', index_col=0)

In [129]:
ct_vars = img_vars.filter(regex='CT')
pet_vars = img_vars.filter(regex='PET')
shape_vars = img_vars.filter(regex='shape')

### Disease-Free Survival

**CT**

In [121]:
ct_dfs_img_ranks = spearmans_rank(ct_vars, dfs)

In [125]:
# Precentage of radiomics features singificantly associated with DFS.
len(ct_dfs_img_ranks.keys()) / ct_vars.shape[1] * 100

35.16483516483517

In [139]:
list(ct_dfs_img_ranks.keys())[:5]

['CT original_glszm_GrayLevelVariance',
 'CT original_glszm_HighGrayLevelZoneEmphasis',
 'CT original_glszm_LargeAreaEmphasis',
 'CT original_glszm_LargeAreaHighGrayLevelEmphasis',
 'CT original_glszm_LargeAreaLowGrayLevelEmphasis']

In [123]:
list(ct_dfs_img_ranks.values())[:5]

[(0.08478873052306143, -0.040167898264550174),
 (0.08813250315089713, -0.021547195404113942),
 (0.09023352233996154, 0.25126976091925657),
 (0.09023352233996154, 0.2399342829830495),
 (0.12370438984333879, 0.2505140623901761)]

**PET**

In [126]:
pet_dfs_img_ranks = spearmans_rank(pet_vars, dfs)

In [127]:
# Precentage of radiomics features singificantly associated with DFS.
len(pet_dfs_img_ranks.keys()) / pet_vars.shape[1] * 100

80.0

In [136]:
list(pet_dfs_img_ranks.keys())[:5]

['PET original_glcm_ClusterProminence',
 'PET original_glcm_ClusterShade',
 'PET original_glcm_ClusterTendency',
 'PET original_glcm_Contrast',
 'PET original_glcm_Correlation']

In [137]:
list(pet_dfs_img_ranks.values())[:5]

[(0.058838762955310826, -0.012846874994368004),
 (0.06797794036032029, -0.013035799626638122),
 (0.08240258670438406, 0.01152440256847718),
 (0.1088439215341862, -0.09257306981235768),
 (0.12435608813173646, 0.05629954041649508)]

**Shape**

In [130]:
shape_dfs_img_ranks = spearmans_rank(shape_vars, dfs)

In [132]:
# Precentage of radiomics features singificantly associated with DFS.
len(shape_dfs_img_ranks.keys()) / shape_vars.shape[1] * 100

15.384615384615385

In [135]:
list(shape_dfs_img_ranks.keys())[:5]

['original_shape_SurfaceVolumeRatio', 'original_shape_Volume']

In [140]:
list(shape_dfs_img_ranks.values())[:5]

[(0.30474459498356976, -0.15869669110689885),
 (0.3771465724399866, 0.22926021847280806)]

### Locoregional Relapse

**CT**

In [143]:
ct_lrr_img_ranks = spearmans_rank(ct_vars, lrr)

In [144]:
# Precentage of radiomics features singificantly associated with DFS.
len(ct_lrr_img_ranks.keys()) / ct_vars.shape[1] * 100

35.16483516483517

In [145]:
list(ct_lrr_img_ranks.keys())[:5]

['CT original_firstorder_10Percentile',
 'CT original_glszm_HighGrayLevelZoneEmphasis',
 'CT original_glszm_LargeAreaEmphasis',
 'CT original_glszm_LargeAreaHighGrayLevelEmphasis',
 'CT original_glszm_LargeAreaLowGrayLevelEmphasis']

In [146]:
list(ct_lrr_img_ranks.values())[:5]

[(0.0012099980431762624, 0.14036125951266576),
 (0.07307029215506046, -0.023660237241525797),
 (0.08756527026261585, 0.2218547406009823),
 (0.1205342473780193, 0.2236975580125271),
 (0.14435025315633293, 0.2071122013086235)]

**PET**

In [147]:
pet_lrr_img_ranks = spearmans_rank(pet_vars, lrr)

In [148]:
# Precentage of radiomics features singificantly associated with DFS.
len(pet_lrr_img_ranks.keys()) / pet_vars.shape[1] * 100

87.36842105263159

In [149]:
list(pet_lrr_img_ranks.keys())[:5]

['PET original_firstorder_10Percentile',
 'PET original_firstorder_90Percentile',
 'PET original_firstorder_Energy',
 'PET original_firstorder_Entropy',
 'PET original_firstorder_Uniformity']

In [150]:
list(pet_lrr_img_ranks.values())[:5]

[(0.0028733005317349287, 0.013206858116071394),
 (0.0031637450300710216, 0.06496776515707492),
 (0.003754210854344388, 0.19236966201626476),
 (0.00456743125226722, -0.01627822046864614),
 (0.01885709230344657, 0.013411615606243044)]

**Shape**

In [151]:
shape_lrr_img_ranks = spearmans_rank(shape_vars, lrr)

In [152]:
# Precentage of radiomics features singificantly associated with DFS.
len(shape_lrr_img_ranks.keys()) / shape_vars.shape[1] * 100

15.384615384615385

In [153]:
list(shape_lrr_img_ranks.keys())[:5]

['original_shape_SurfaceVolumeRatio', 'original_shape_Volume']

In [154]:
list(shape_lrr_img_ranks.values())[:5]

[(0.130596755487678, -0.1364708671994044),
 (0.5257739997249165, 0.19462214484671708)]

## Clinical Variables

In [155]:
clin_vars = pd.read_csv('./../../data/to_analysis/clinical_params.csv', index_col=0)

In [156]:
dfs_clin_ranks = spearmans_rank(clin_vars, dfs)

### Disease-Free Survival

In [157]:
# Precentage of radiomics features singificantly associated with DFS.
len(dfs_clin_ranks.keys()) / clin_vars.shape[1] * 100

92.85714285714286

In [160]:
list(dfs_clin_ranks.keys())[:5]

['Age', 'Naxogin Days', 'ICD-10_C03', 'ICD-10_C04', 'ICD-10_C05']

In [161]:
list(dfs_clin_ranks.values())[:5]

[(0.00254323068389186, 0.10353073849748919),
 (0.007168098774324121, 0.08715493501002772),
 (0.020901280668800013, -0.04923846152029757),
 (0.020901280668800013, 0.0026787422721745347),
 (0.05520975423222327, -0.09923542145894367)]

### Locoregional Relapse

In [158]:
lrr_clin_ranks = spearmans_rank(clin_vars, lrr)

In [159]:
# Precentage of radiomics features singificantly associated with LRR.
len(lrr_clin_ranks.keys()) / clin_vars.shape[1] * 100

100.0

In [163]:
list(lrr_clin_ranks.keys())[:5]

['Age', 'Years Smoking', 'Naxogin Days', 'Sex_M', 'ICD-10_C02']

In [164]:
list(lrr_clin_ranks.values())[:5]

[(0.011389244772878017, 0.0944956182357087),
 (0.013262504114678022, 0.0329684733221144),
 (0.024715673706297233, -0.011730198610754186),
 (0.0811751318295311, 0.09088023228246712),
 (0.0811751318295311, 0.17950982180079614)]

**Observations**
* TODO: Compare percentage of associations + strongest features, rank and p-values.